<div align="center">
    <img src="images/logo_fmkn.png" alt="logo_fmkn" />
</div>

# Машинное обучение 1

### Лекция 13. ЕМ алгоритм

<br />
<br />
10 декабря 2021

### Пятиминутка

1. Выпишите формулу апостериорной вероятности параметров, иллюстрирующую байесовский подход к задаче машинного обучения
2. Выпишите определение сопряжённых распределений
3. Перечислите основные достоинства байесовского подхода

<div align="center">
    <img src="images/kl_div_mem.jpg" alt="kl_div_mem" />
</div>

## Расстояние Кульбака-Лейблера (KL-дивергенция)

### Неравенство Йенсена

Если $f(x)$ — вогнутая (выпуклая вверх) функция, то справедливо неравенство Йенсена:

$$ f(wx_1 + (1 − w)x_2) \geq \color{orange}{wf(x_1) + (1 − w)f(x_2)} $$

<div align="center">
    <img src="images/concave_function.jpg" alt="concave_function" />
</div>

Либо в более общем случае

$$f(w_1x_1 + w_2x_2 + \dots + w_nx_n) \geq w_1f(x_1) + w_2f(x_2) + \dots + w_nf(x_n)$$

где $ \sum\limits_{i=1}^n w_i = 1$

Можно доказать, что справедливо и

$$  f\left(\mathbb{E}_{p(x)} u(x)\right) \geq \mathbb{E}_{p(x)} f(u(x)) $$


### Расстояние Кульбака-Лейблера между вероятностными распределениями

$$ KL(p||q) = \int p(x) log \frac{p(x)}{q(x)} dx $$

Свойства дивергенции Кульбака-Лейблера:
 * $KL(p||q) \neq KL(q||p)$ — несимметричность
 * $KL(p||q) \geq 0$ — неотрицательность
 * $KL(p||q) = 0 \Leftrightarrow q(x) = p(x)$


### Доказательство неотрицательности KL-дивергенции

$$ -KL(p||q) = -\int p(x) \log\left(\frac{p(x)}{q(x)}\right)dx = \int p(x) \log\left(\frac{q(x)}{p(x)}\right)dx \leq \\
   \leq \log\left( \int p(x) \frac{q(x)}{p(x)}dx\right) = \log\left( \int q(x) dx\right) = \log 1 = 0$$



### Несимметричность KL-дивергенции. Пример: поиск похожего распределения.

$ KL(? || ?) $ — на какое место поставить $p$?

<div align="center">
    <img src="images/example.jpg" alt="example_1" width=400 />
</div>


### Несимметричность KL-дивергенции. Первым аргументом: mass covering

$ KL(p || q) = \int p(x) \log \frac{p(x)}{q(x)} dx$

<div align="center">
    <img src="images/example_1.jpg" alt="example_1" width=400 />
</div>


### Несимметричность KL-дивергенции. Вторым аргументом: mode collapsing

$ KL(q || p) = \int q(x) \log \frac{q(x)}{p(x)} dx$

<div align="center">
    <img src="images/example_2.jpg" alt="example_2" width=400 />
</div>


### Напоминания и экспоненциальное семейство распределений

 * предполагаем, что в линейной регрессии ответы генерируются из нормального распределения
 
 $$ y_i \sim N(\mu_i, \sigma_i^2),\ \ \ \mu_i = E y_i = x_i^T w$$
 
 * Обобщенная линейная модель для регрессии

 $$ y_i \sim \color{red}{Exp}(\mu_i, \phi_i),\ \ \ \mu_i = E y_i,\ \ \ \color{red}{g(\mu_i)} = \theta_i = g(x_i^T w)$$

 $g(\mu_i)$ — монотонная функция связи (link function)
 
 $Exp$ — экспоненциальное семейство распределений с параметрами $\theta_i, \phi_i$ и параметрами-функциями $c(\theta), h(y, \phi)$, то есть с плотностью
 
 $$ p(y_i | \theta_i, \phi_i) = \exp\left(\frac{y_i\theta_i - c(\theta_i)}{\phi_i} + h(y_i, \phi_i)\right) \\
    \theta_i = g\left(\sum\limits_{j=1}^n w_j f_j(x_i) \right)
 $$
 
 Обычно удобно работать с семейством распределений.

<div align="center">
    <img src="images/you-wont-be-disappointed-if-you-have-no-expectation.jpg" alt="exp_mem" />
</div>

### EM-алгоритм: мотивирующий пример

Как научиться восстанавливать распределения?

Гауссиана:

<div align="center">
    <img src="images/gauss_1.jpg" alt="gauss_1" width=400 />
</div>

Несколько гауссиан:

<div align="center">
    <img src="images/gauss_3.jpg" alt="gauss_1" width=400 />
</div>


Неизвестное распределение:

<div align="center">
    <img src="images/gauss_unknown.jpg" alt="gauss_unknown" width=400 />
</div>

Можем ввести номер гауссианы и свести задачу к уже решенной!


### Вводим латентные переменные

Хотим $p(X|\theta) \underset{\theta}{\to} \max$ (неполное правдоподобие)

Умеем $p(X, Z|\theta) \underset{\theta}{\to} \max$ (полное правдоподобие)

Пусть в предыдущем примере $z_i$ — это идентификатор гауссианы, из которой берётся $i$-ый объект.


### Вариационная нижняя граница

Для одного наблюдения $x$:

$$ \log p(x|\theta) = \int q(z) \log p(x|\theta) dz = \int q(z) \log \frac{p(x, z|\theta)}{p(z|x,\theta)} dz = \\ 
   \int q(z) \log \frac{p(x, z|\theta)q(z)}{p(z|x,\theta)q(z)} dz = 
   \int q(z) \log \frac{p(x, z|\theta)}{q(z)} dz + \int q(z) \log \frac{q(z)}{p(z|x,\theta)} dz \geq \\
   \geq \int q(z) \log \frac{p(x, z|\theta)}{q(z)} dz = \mathscr{L}(q, \theta)
$$
— вариационная нижняя граница (просто отбросили неотрицательное $KL(q(z) || p(z|x,\theta))$)

Причем равенство достигается при $q^* = p(z| x, \theta)$

Для выборки $X$:

$$ \log p(X| \theta) = \sum\limits_{i=1}^n \log p(x_i|\theta) \geq \sum\limits_{i=1}^n \mathscr{L}(q_i, \theta) \underset{q, \theta}{\to} \max$$

Максимизировать по $\theta$ легко, если знаем $q$ и наоборот.


### EM-алгоритм

E-шаг (Expectation):

$$ \sum\limits_{i=1}^n \mathscr{L}(q_i, \theta) \underset{q_i}{\to} \max \Leftrightarrow
 q_i(z_i) = p(z_i|x_i, \theta)
$$

M-шаг (Maximization):

$$ \max\limits_\theta \sum\limits_{i=1}^n \mathscr{L}(q_i, \theta) = 
   \max\limits_\theta \sum\limits_{i=1}^n \int q_i(z_i) \log\frac{p(x_i, z_i| \theta)}{q_i(z_i)} dz_i = \\
   \max\limits_\theta \sum\limits_{i=1}^n E_{q_i(z_i)} \log p(x_i, z_i| \theta)
$$


<div align="center">
    <img src="images/em.gif" alt="em" width=600 />
</div>

### Gaussian Mixture Model

<div align="center">
    <img src="images/GMM.jpg" alt="GMM" width=400 />
</div>

$$ p(x_i, z_i|\theta) = p(x_i|z_i,\theta)p(z_i|\theta) = N(x_i|\mu_{z_i}, \sigma_{z_i}^2) \pi_{z_i} $$


Параметры: $\{\mu_j, \sigma_j, \pi_j\}$, здесь $\pi_j$ — априорная вероятность $j$-ой гауссианы (cкрытая переменная $z_i$)


<div class="alert alert-info">
<b>Вопрос 1:</b> Как найти число компонент смеси?
</div>

 * можно простым перебором по их количеству (метод model selection)
 * можно по-другому – ввести априорное распределение **сразу на все модели разной сложности**. Это и есть основная идея байесовских непараметрических методов
 * см. процесс китайского ресторана и индийского буфета 
 

### EM-алгоритм для Gaussian Mixture Model

E-шаг:

$$ q_i(z_i{\Tiny=}j) = p(z_i{\Tiny=}j|x_i, \theta) = \frac{p(x_i|z_i {\Tiny=} j, \theta)p(z_i {\Tiny=} j|\theta)}{p(x_i|\theta)} = \\
   = \frac{p(x_i|z_i {\Tiny=} j, \theta)p(z_i {\Tiny=} j|\theta)}{\sum\limits_{k=1}^K p(x_i|z_i {\Tiny=} k, \theta)p(z_i {\Tiny=} k|\theta)}
   = \frac{N(x_i|\mu_j, \sigma_j^2) \pi_j}{\sum\limits_{k=1}^K N(x_i|\mu_k, \sigma_k^2) \pi_k}
$$

M-шаг:

$$ \sum\limits_{i=1}^n \mathscr{L}(q,\theta) = \sum\limits_{i=1}^n \sum\limits_{k=1}^K q_i(z_i=k) \log\frac{p(x_i|z_i = k, \theta)p(z_i = k|\theta)}{q_i(z_i=k)} = \\
\sum\limits_{i=1}^n \sum\limits_{k=1}^K q_i(z_i=k) \log\frac{N(x_i|\mu_k, \sigma_k^2)\pi_k}{q_i(z_i=k)}
$$

Далее как обычно находим нули производной:
$$ \mu_j = \frac{\sum\limits_{i=1}^n q_i(z_i=j)x_i}{\sum\limits_{i=1}^n q_i(z_i=j)};\ 
   \sigma_j^2 = \frac{\sum\limits_{i=1}^n q_i(z_i=j)(x_i-\mu_j)^2}{\sum\limits_{i=1}^n q_i(z_i=j)};\ 
   \pi_j = \frac{1}{n} \sum\limits_{i=1}^n q_i(z_i = j)
$$


### Метод главных компонент: постановка задачи

$f_1(x), \dots, f_D(x)$ — исходные числовые признаки

$g_1(x), \dots, g_d(x)$ — новые числовые признаки, $d \leq D$

**Требование**: старые признаки должны линейно восстанавливаться по новым как можно точнее на обучающей выборке $X^n$

$ \hat f_j(x) = \sum\limits_{s=1}^d g_s(x) u_{js}, j=1,\dots,D , \forall x \in X$

$ \sum\limits_{i=1}^n \sum\limits_{j=1}^D (\hat f_j(x_i) - f_j(x_i))^2 \to \min\limits_{\{g_s(x_i)\}, \{u_{js}\}}$


### Матричные обозначения

Матрицы «объекты-признаки», старая и новая:

$F_{n \times D} = 
\left[ \begin{array}{ccc}
   f_1(x_1) & \dots & f_D(x_1) \\
     \dots  & \dots &   \dots  \\
   f_1(x_n) & \dots & f_D(x_n)
  \end{array} \right]
\ \ \ 
G_{n \times d} = 
\left[ \begin{array}{ccc}
   g_1(x_1) & \dots & g_d(x_1) \\
     \dots  & \dots &   \dots  \\
   g_1(x_n) & \dots & g_d(x_n)
  \end{array} \right]
$

Матрица линейного преобразования новых признаков в старые:

$U_{D \times d} =  
\left[ \begin{array}{ccc}
   u_{11} & \dots & u_{1d} \\
     \dots  & \dots &   \dots  \\
   u_{D1} & \dots & u_{Dd}
  \end{array} \right],
  \hat F = GU^T \underset{\text{хотим}}{\approx} F
$

**Найти**: и новые признаки $G$, и преобразование $U$:

$\sum\limits_{i=1}^\ell \sum\limits_{j=1}^n (\hat f_j(x_i) - f_j(x_i))^2 = \| GU^T - F\|^2 \to \min\limits_{G,U}$


### Вероятностная интерпретация метода главных компонент

$$p(x,z|\theta) = p(x|z, \theta)p(z) = N(x|Wz + \mu, \sigma^2I) N(z|0, I)$$

$x \in R^D,\ z \in R^d,\ z \sim N(0, I)$

Параметры ${W, \mu, \sigma}$

$\mu \in R^D, W \in R^{D\times d}$


<div class="alert alert-info">
<b>Замечание</b> На матрицу $W$ тоже можно наложить ограничения, выраженные априорным распределением, чтобы модель минимизировала $d$ при оптимизации.
</div>

### EM-алгоритм для PPCA

E-шаг (считается аналитически, т.к. сопряженные распределения):

$$q_i(z_i) = p(z_i|x_i, \theta) = \frac{p(x_i|z_i, \theta)p(z_i)}{\int p(x_i|z_i, \theta)p(z_i) dz_i} = \\
  = N(z_i| (\sigma^2I + W^TW)^{-1}W^T (x_i - \mu), (I + \sigma^{-2}W^TW)^{-1})
$$


M-шаг:
$$E_Z\log p(X, Z|\theta) = \sum\limits_{i=1}^n E_{z_i} \log p(x_i, z_i|\theta) = \\
  \sum\limits_{i=1}^n E_{z_i} \log p(x_i| z_i, \theta) + const \underset{\theta}{\to} \max
$$


### M-шаг, пример: поиск W

$$\sum\limits_{i=1}^n E_{z_i} \log p(x_i| z_i, \theta) = \\
 = \sum\limits_{i=1}^n E_{z_i} \left[-\frac{D}{2}\log2\pi-D\log\sigma-\frac{1}{2\sigma^2}\left((x_i - \mu)-Wz_i\right)^T\left((x_i - \mu)-Wz_i\right) \right] = \\
 = const(\sigma) - \frac{1}{2\sigma^2} \sum\limits_{i=1}^n E_{z_i} \left[(x_i - \mu)^T(x_i - \mu) - 2(x_i - \mu)^T Wz_i + z_i^TW^T Wz_i \right] = \\
 = const(\sigma) - \frac{1}{2\sigma^2} \sum\limits_{i=1}^n \left[(x_i - \mu)^T(x_i - \mu) - 2(x_i - \mu)^T W E_{z_i}z_i + tr(W^T WE_{z_i}z_iz_i^T) \right]
$$

Далее дифференцируем по $W$:

$$ - \frac{1}{2\sigma^2} \sum\limits_{i=1}^n \left[-2(x_i - \mu) E_{z_i} z_i^T + 2W E_{z_i} z_i z_i^T \right] = 0 \\
  W^* = \left( \sum\limits_{i=1}^n (x_i - \mu) E_{z_i} z_i^T \right) \left( \sum\limits_{i=1}^n E_{z_i} z_i z_i^T \right)^{-1}
$$

Матожидания мы можем посчитать, т.к. знаем распределение $q$ с предыдущего Е-шага.


### Сравнение оценок на время выполнения

При $n > D > d$

Стандартный PCA: $O(nD^2)$

1 итерация PPCA: $O(nDd)$

Обычно нужно 10-100 итераций для сходимости, поэтому может быть быстрее стандартного.

### Случай пропущенных признаков в PPCA

$$(X, Z) = X_{\text{obs}} \cup X_{\text{hid}} \\
   P(X_{\text{obs}}|\theta) \underset{\theta}{\to} \max
$$

E-шаг

$$ q(X_{\text{hid}}) = p(X_{\text{hid}}|X_{\text{obs}}, \theta) $$


M-шаг

$$E_{X_\text{hid}} \log p(X_{\text{obs}} , X_{\text{hid}} |\theta) \underset{\theta}{\to} \max $$

 * ещё получаем подходящие вероятностные распределения для скрытых переменных


### Резюме

 * Частая метрика между вероятностными распределениями — расстояние Кульбака-Лейблера (KL-дивергенция)
 * ЕМ-алгоритм позволяет итеративно решать запутанные задачи, разделяя их на два шага
 * Вероятностный метод главных компонент с использованием ЕМ-алгоритма